# End to end

In [25]:
timestamp_prefix = 'automated-job'

In [32]:
import sagemaker
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

timestamp_prefix = 'automated-job'

prefix = 'sagemaker/spark-preprocess-demo/' + timestamp_prefix
input_prefix = prefix + '/input/raw/abalone'
input_preprocessed_prefix = prefix + '/input/preprocessed/abalone'
model_prefix = prefix + '/model'
#Jay Change
mleap_model_prefix = prefix + '/mleap-model'

In [33]:
# Fetch the dataset from the SageMaker bucket
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/abalone/abalone.csv

# Uploading the training data to S3
sagemaker_session.upload_data(path='abalone.csv', bucket=bucket, key_prefix=input_prefix)

--2020-08-19 14:52:26--  https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/abalone/abalone.csv
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.209.216
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.209.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191873 (187K) [binary/octet-stream]
Saving to: ‘abalone.csv.2’

abalone.csv.2       100%[===================>] 187.38K   866KB/s    in 0.2s    

2020-08-19 14:52:26 (866 KB/s) - ‘abalone.csv.2’ saved [191873/191873]



's3://sagemaker-us-east-1-452432741922/sagemaker/spark-preprocess-demo/automated-job/input/raw/abalone/abalone.csv'

In [5]:
%cd /home/ec2-user/SageMaker/sagemaker-ml-workflow-with-apache-airflow/container
!docker build -t sagemaker-spark-example .
%cd ../

/home/ec2-user/SageMaker/sagemaker-ml-workflow-with-apache-airflow/container
Sending build context to Docker daemon  44.03kB
Step 1/32 : FROM openjdk:8-jre-slim
8-jre-slim: Pulling from library/openjdk

52930446: Pulling fs layer 
9b8e633f: Pulling fs layer 
86d6fc62: Pulling fs layer 
Digest: sha256:c8740afc9ec89f879bdb3d02cb885a8c3fff565f2f850020127194765922b631[1A
Status: Downloaded newer image for openjdk:8-jre-slim
 ---> d2f9f3c77c25
Step 2/32 : RUN apt-get update
 ---> Running in 4a5a2ac05cba
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [51.9 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [220 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7906 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [7868 B]
Fetched 8374 kB in 1s (5662 kB/s)
Rea

Get:24 http://deb.debian.org/debian buster/main amd64 python3-minimal amd64 3.7.3-1 [36.6 kB]
Get:25 http://deb.debian.org/debian buster/main amd64 libmpdec2 amd64 2.4.2-2 [87.2 kB]
Get:26 http://deb.debian.org/debian buster/main amd64 libpython3.7-stdlib amd64 3.7.3-2+deb10u2 [1732 kB]
Get:27 http://deb.debian.org/debian buster/main amd64 python3.7 amd64 3.7.3-2+deb10u2 [330 kB]
Get:28 http://deb.debian.org/debian buster/main amd64 libpython3-stdlib amd64 3.7.3-1 [20.0 kB]
Get:29 http://deb.debian.org/debian buster/main amd64 python3 amd64 3.7.3-1 [61.5 kB]
Get:30 http://deb.debian.org/debian buster/main amd64 netbase all 5.6 [19.4 kB]
Get:31 http://deb.debian.org/debian buster/main amd64 bzip2 amd64 1.0.6-9.2~deb10u1 [48.4 kB]
Get:32 http://deb.debian.org/debian buster/main amd64 libapparmor1 amd64 2.13.2-10 [94.7 kB]
Get:33 http://deb.debian.org/debian buster/main amd64 libdbus-1-3 amd64 1.12.20-0+deb10u1 [215 kB]
Get:34 http://deb.debian.org/debian buster/main amd64 dbus amd64 1.12

Get:118 http://deb.debian.org/debian buster/main amd64 libpython2.7 amd64 2.7.16-2+deb10u1 [1036 kB]
Get:119 http://deb.debian.org/debian buster/main amd64 libpython2.7-dev amd64 2.7.16-2+deb10u1 [31.6 MB]
Get:120 http://deb.debian.org/debian buster/main amd64 libpython2-dev amd64 2.7.16-1 [20.9 kB]
Get:121 http://deb.debian.org/debian buster/main amd64 libpython-dev amd64 2.7.16-1 [20.9 kB]
Get:122 http://deb.debian.org/debian buster/main amd64 libpython3.7 amd64 3.7.3-2+deb10u2 [1498 kB]
Get:123 http://deb.debian.org/debian buster/main amd64 libpython3.7-dev amd64 3.7.3-2+deb10u2 [48.4 MB]
Get:124 http://deb.debian.org/debian buster/main amd64 libpython3-dev amd64 3.7.3-1 [20.1 kB]
Get:125 http://deb.debian.org/debian buster/main amd64 libsasl2-modules amd64 2.1.27+dfsg-1+deb10u1 [104 kB]
Get:126 http://deb.debian.org/debian buster/main amd64 libxml2 amd64 2.9.4+dfsg1-7+b3 [687 kB]
Get:127 http://deb.debian.org/debian buster/main amd64 manpages-dev all 4.16-2 [2232 kB]
Get:128 http:/

Selecting previously unselected package python3-minimal.
(Reading database ... 9963 files and directories currently installed.)
Preparing to unpack .../python3-minimal_3.7.3-1_amd64.deb ...
Unpacking python3-minimal (3.7.3-1) ...
Selecting previously unselected package libmpdec2:amd64.
Preparing to unpack .../libmpdec2_2.4.2-2_amd64.deb ...
Unpacking libmpdec2:amd64 (2.4.2-2) ...
Selecting previously unselected package libpython3.7-stdlib:amd64.
Preparing to unpack .../libpython3.7-stdlib_3.7.3-2+deb10u2_amd64.deb ...
Unpacking libpython3.7-stdlib:amd64 (3.7.3-2+deb10u2) ...
Selecting previously unselected package python3.7.
Preparing to unpack .../python3.7_3.7.3-2+deb10u2_amd64.deb ...
Unpacking python3.7 (3.7.3-2+deb10u2) ...
Selecting previously unselected package libpython3-stdlib:amd64.
Preparing to unpack .../libpython3-stdlib_3.7.3-1_amd64.deb ...
Unpacking libpython3-stdlib:amd64 (3.7.3-1) ...
Setting up python3-minimal (3.7.3-1) ...
Selecting previously unselected package pyt

Unpacking libkrb5support0:amd64 (1.17-3) ...
Selecting previously unselected package libk5crypto3:amd64.
Preparing to unpack .../047-libk5crypto3_1.17-3_amd64.deb ...
Unpacking libk5crypto3:amd64 (1.17-3) ...
Selecting previously unselected package libkrb5-3:amd64.
Preparing to unpack .../048-libkrb5-3_1.17-3_amd64.deb ...
Unpacking libkrb5-3:amd64 (1.17-3) ...
Selecting previously unselected package libgssapi-krb5-2:amd64.
Preparing to unpack .../049-libgssapi-krb5-2_1.17-3_amd64.deb ...
Unpacking libgssapi-krb5-2:amd64 (1.17-3) ...
Selecting previously unselected package libsasl2-modules-db:amd64.
Preparing to unpack .../050-libsasl2-modules-db_2.1.27+dfsg-1+deb10u1_amd64.deb ...
Unpacking libsasl2-modules-db:amd64 (2.1.27+dfsg-1+deb10u1) ...
Selecting previously unselected package libsasl2-2:amd64.
Preparing to unpack .../051-libsasl2-2_2.1.27+dfsg-1+deb10u1_amd64.deb ...
Unpacking libsasl2-2:amd64 (2.1.27+dfsg-1+deb10u1) ...
Selecting previously unselected package libldap-common.
P

Selecting previously unselected package libpython3-dev:amd64.
Preparing to unpack .../095-libpython3-dev_3.7.3-1_amd64.deb ...
Unpacking libpython3-dev:amd64 (3.7.3-1) ...
Selecting previously unselected package libsasl2-modules:amd64.
Preparing to unpack .../096-libsasl2-modules_2.1.27+dfsg-1+deb10u1_amd64.deb ...
Unpacking libsasl2-modules:amd64 (2.1.27+dfsg-1+deb10u1) ...
Selecting previously unselected package libxml2:amd64.
Preparing to unpack .../097-libxml2_2.9.4+dfsg1-7+b3_amd64.deb ...
Unpacking libxml2:amd64 (2.9.4+dfsg1-7+b3) ...
Selecting previously unselected package manpages-dev.
Preparing to unpack .../098-manpages-dev_4.16-2_all.deb ...
Unpacking manpages-dev (4.16-2) ...
Selecting previously unselected package publicsuffix.
Preparing to unpack .../099-publicsuffix_20190415.1030-1_all.deb ...
Unpacking publicsuffix (20190415.1030-1) ...
Selecting previously unselected package python2.7-dev.
Preparing to unpack .../100-python2.7-dev_2.7.16-2+deb10u1_amd64.deb ...
Unpacki

invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up xz-utils (5.2.4-1) ...
update-alternatives: using /usr/bin/xz to provide /usr/bin/lzma (lzma) in auto mode
update-alternatives: warning: skip creation of /usr/share/man/man1/lzma.1.gz because associated file /usr/share/man/man1/xz.1.gz (of link group lzma) doesn't exist
update-alternatives: warning: skip creation of /usr/share/man/man1/unlzma.1.gz because associated file /usr/share/man/man1/unxz.1.gz (of link group lzma) doesn't exist
update-alternatives: warning: skip creation of /usr/share/man/man1/lzcat.1.gz because associated file /usr/share/man/man1/xzcat.1.gz (of link group lzma) doesn't exist
update-alternatives: warning: skip creation of /usr/share/man/man1/lzmore.1.gz because associated file /usr/share/man/man1/xzmore.1.gz (of link group lzma) doesn't exist
update-alternatives: warning: skip creation of /usr/share/man/man1/lzless.1.gz because associated file /usr/sh

  Running setup.py bdist_wheel for psutil: started
  Running setup.py bdist_wheel for psutil: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/33/48/b6/72b7243c5caf65b7d5b460e9fad82b1256992284e870b7db59
  Running setup.py bdist_wheel for mleap: started
  Running setup.py bdist_wheel for mleap: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/bc/a5/8d/7d021f2741ed8a7354e4fe19c1890e71ed719d681cd16cf292
  Running setup.py bdist_wheel for nose-exclude: started
  Running setup.py bdist_wheel for nose-exclude: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/fe/b9/8f/764ed47b11e2e062ae8fe5f09d1c801a600292fdd9ba3477bb
Successfully built psutil mleap nose-exclude
Removing intermediate container 6fb2cb481ad4
 ---> be02c955870a
Step 5/32 : RUN apt-get clean
 ---> Running in f20ab6ec322b
Removing intermediate container f20ab6ec322b
 ---> aeb959a49476
Step 6/32 : RUN rm -rf /var/lib/apt/lists/*
 ---> Running in 52c7ae802

 ---> Running in db8df67399df
Removing intermediate container db8df67399df
 ---> 6bd79aea9957
Step 27/32 : ENV HDFS_SECONDARYNAMENODE_USER="root"
 ---> Running in 936983e5726b
Removing intermediate container 936983e5726b
 ---> 982e2f9817ed
Step 28/32 : COPY program /opt/program
 ---> 021520dccaee
Step 29/32 : RUN chmod +x /opt/program/submit
 ---> Running in 153d12754051
Removing intermediate container 153d12754051
 ---> 62a484f6ee9b
Step 30/32 : COPY hadoop-config /opt/hadoop-config
 ---> 368c07a96425
Step 31/32 : WORKDIR $SPARK_HOME
 ---> Running in b23a7d0b1df0
Removing intermediate container b23a7d0b1df0
 ---> 4bb53f53a8a9
Step 32/32 : ENTRYPOINT ["/opt/program/submit"]
 ---> Running in f1941e83d967
Removing intermediate container f1941e83d967
 ---> 490cdd58cf2a
Successfully built 490cdd58cf2a
Successfully tagged sagemaker-spark-example:latest
/home/ec2-user/SageMaker/sagemaker-ml-workflow-with-apache-airflow


In [7]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

ecr_repository = 'sagemaker-spark-example'
tag = ':latest'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
spark_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

# Create ECR repository and push docker image
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $spark_repository_uri
!docker push $spark_repository_uri

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-spark-example' already exists in the registry with id '452432741922'
The push refers to repository [452432741922.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark-example]

622d5b24: Preparing 
7a6f14a8: Preparing 
51027859: Preparing 
29112dff: Preparing 
b9b3e8c6: Preparing 
ddc8bb51: Preparing 
6d71e867: Preparing 
dee7661f: Preparing 
2bbb24bf: Preparing 
47855bc1: Preparing 
8102614d: Preparing 
52ed4cbd: Preparing 
0fa5728d: Preparing 
2bbb24bf: Pushed   490.3MB/481.8MBlatest: digest: sha256:0c10dc66c37a172547ed295c52a03efbf9847fa881fa25a3f

In [15]:
spark_repository_uri

'452432741922.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark-example:latest'

In [8]:
%%writefile preprocess.py
from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv

import boto3
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import StructField, StructType, StringType, DoubleType
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.functions import *
from mleap.pyspark.spark_support import SimpleSparkSerializer

def csv_line(data):
    r = ','.join(str(d) for d in data[1])
    return str(data[0]) + "," + r


def main():
    spark = SparkSession.builder.appName("PySparkAbalone").getOrCreate()
    
    # Convert command line args into a map of args
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))
    
    # This is needed to save RDDs which is the only way to write nested Dataframes into CSV format
    spark.sparkContext._jsc.hadoopConfiguration().set("mapred.output.committer.class",
                                                      "org.apache.hadoop.mapred.FileOutputCommitter")
    
    # Defining the schema corresponding to the input data. The input data does not contain the headers
    schema = StructType([StructField("sex", StringType(), True), 
                         StructField("length", DoubleType(), True),
                         StructField("diameter", DoubleType(), True),
                         StructField("height", DoubleType(), True),
                         StructField("whole_weight", DoubleType(), True),
                         StructField("shucked_weight", DoubleType(), True),
                         StructField("viscera_weight", DoubleType(), True), 
                         StructField("shell_weight", DoubleType(), True), 
                         StructField("rings", DoubleType(), True)])

    # Downloading the data from S3 into a Dataframe
    total_df = spark.read.csv(('s3a://' + os.path.join(args['s3_input_bucket'], args['s3_input_key_prefix'],
                                                   'abalone.csv')), header=False, schema=schema)

    #StringIndexer on the sex column which has categorical value
    sex_indexer = StringIndexer(inputCol="sex", outputCol="indexed_sex")
    
    #one-hot-encoding is being performed on the string-indexed sex column (indexed_sex)
    sex_encoder = OneHotEncoder(inputCol="indexed_sex", outputCol="sex_vec")

    #vector-assembler will bring all the features to a 1D vector for us to save easily into CSV format
    assembler = VectorAssembler(inputCols=["sex_vec", 
                                           "length", 
                                           "diameter", 
                                           "height", 
                                           "whole_weight", 
                                           "shucked_weight", 
                                           "viscera_weight", 
                                           "shell_weight"], 
                                outputCol="features")
    
    # The pipeline comprises of the steps added above
    pipeline = Pipeline(stages=[sex_indexer, sex_encoder, assembler])
    
    # This step trains the feature transformers
    model = pipeline.fit(total_df)
    
    # This step transforms the dataset with information obtained from the previous fit
    transformed_total_df = model.transform(total_df)
    
    # Split the overall dataset into 80-20 training and validation
    (train_df, validation_df) = transformed_total_df.randomSplit([0.8, 0.2])
    
    # Convert the train dataframe to RDD to save in CSV format and upload to S3
    train_rdd = train_df.rdd.map(lambda x: (x.rings, x.features))
    train_lines = train_rdd.map(csv_line)
    train_lines.saveAsTextFile('s3a://' + os.path.join(args['s3_output_bucket'], args['s3_output_key_prefix'], 'train'))
    
    # Convert the validation dataframe to RDD to save in CSV format and upload to S3
    validation_rdd = validation_df.rdd.map(lambda x: (x.rings, x.features))
    validation_lines = validation_rdd.map(csv_line)
    validation_lines.saveAsTextFile('s3a://' + os.path.join(args['s3_output_bucket'], args['s3_output_key_prefix'], 'validation'))
    
    # Serialize and store the model via MLeap  
    SimpleSparkSerializer().serializeToBundle(model, "jar:file:/opt/ml/model.zip", validation_df)    
    # Unzip the model as SageMaker expects a .tar.gz file but MLeap produces a .zip file
    import zipfile
    with zipfile.ZipFile("/opt/ml/model.zip") as zf:
        zf.extractall("/opt/ml/model")

    # Writw back the content as a .tar.gz file
    import tarfile
    with tarfile.open("/opt/ml/model.tar.gz", "w:gz") as tar:
        tar.add("/opt/ml/model/bundle.json", arcname='bundle.json')
        tar.add("/opt/ml/model/root", arcname='root')
    
    # Upload the model in tar.gz format to S3 so that it can be used with SageMaker for inference later
    s3 = boto3.resource('s3') 
    file_name = os.path.join(args['s3_mleap_model_prefix'], 'model.tar.gz')
    s3.Bucket(args['s3_model_bucket']).upload_file('/opt/ml/model.tar.gz', file_name)    

if __name__ == "__main__":
    main()

Writing preprocess.py


In [9]:
! aws s3 cp preprocess.py s3://ml-lab-mggaska/sparkdemo/preprocess.py

upload: ./preprocess.py to s3://ml-lab-mggaska/sparkdemo/preprocess.py


In [10]:

from sagemaker.processing import ScriptProcessor, ProcessingInput
spark_processor = ScriptProcessor(base_job_name='spark-preprocessor',
                                  image_uri=spark_repository_uri,
                                  command=['/opt/program/submit'],
                                  role=role,
                                  instance_count=2,
                                  instance_type='ml.r5.xlarge',
                                  max_runtime_in_seconds=800,
                                  env={'mode': 'python'})

spark_processor.run(code='preprocess.py',
                    arguments=['s3_input_bucket', bucket,
                               's3_input_key_prefix', input_prefix,
                               's3_output_bucket', bucket,
                               's3_output_key_prefix', input_preprocessed_prefix,
                               's3_model_bucket', bucket,
                               's3_mleap_model_prefix', mleap_model_prefix],
                    logs=True)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  spark-preprocessor-2020-08-19-13-27-41-130
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-452432741922/spark-preprocessor-2020-08-19-13-27-41-130/input/code/preprocess.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  []
2020-08-19 13:31:17,065 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = algo-1/10.0.126.24
STARTUP_MSG:   args = [-format, -force]
STARTUP_MSG:   version = 3.0.0
STARTUP_MSG:   classpath = /usr/hadoop-3.0.0/etc/hadoop:/usr/hadoop-3.0.0/share/hadoop/common/lib/kerby-xdr-1.0.1.jar:/usr/hadoop-3.0.0/share/hadoop/common/lib/commons-configuration2-2.1.1.jar:/usr/hadoop-3.0.0/share/hadoop/common/lib/jackson-annotations-2.7.8.jar:/usr/hadoop-3.0.0/share/hadoop/common/lib/commons-codec-1.4.jar:/us

Starting nodemanagers
localhost: /usr/hadoop-3.0.0/bin/../libexec/hadoop-functions.sh: line 981: ssh: command not found
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/spark-2.2.0/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
ml.combust.mleap#mleap-spark_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent;1.0
#011confs: [default]
#011found ml.combust.mleap#mleap-spark_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-spark-base_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-runtime_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-core_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-base_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-tensor_2.11;0.8.1 in central
#011found io.spray#spray-json_2.11;1.3.2 in central
#011found org.apache.spark#spark-mllib-local_2.11;2.2.0 in central
#011found org.scalanlp#b

2020-08-19 13:31:38,041 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
2020-08-19 13:31:38,725 INFO yarn.Client: Uploading resource file:/tmp/spark-f4563d98-db77-48d6-a62f-10fe21f8fbb7/__spark_libs__8511590895264708562.zip -> hdfs://10.0.126.24/user/root/.sparkStaging/application_1597843887193_0001/__spark_libs__8511590895264708562.zip
2020-08-19 13:31:40,228 INFO yarn.Client: Uploading resource file:/root/.ivy2/jars/ml.combust.mleap_mleap-spark_2.11-0.8.1.jar -> hdfs://10.0.126.24/user/root/.sparkStaging/application_1597843887193_0001/ml.combust.mleap_mleap-spark_2.11-0.8.1.jar
2020-08-19 13:31:40,659 INFO yarn.Client: Uploading resource file:/root/.ivy2/jars/ml.combust.mleap_mleap-spark-base_2.11-0.8.1.jar -> hdfs://10.0.126.24/user/root/.sparkStaging/application_1597843887193_0001/ml.combust.mleap_mleap-spark-base_2.11-0.8.1.jar
2020-08-19 13:31:41,084 INFO yarn.Client: Uploading resource file:/root/.ivy

2020-08-19 13:32:06,174 INFO cluster.YarnClientSchedulerBackend: SchedulerBackend is ready for scheduling beginning after waiting maxRegisteredResourcesWaitingTime: 30000(ms)
2020-08-19 13:32:06,338 INFO internal.SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/usr/spark-2.2.0/spark-warehouse').
2020-08-19 13:32:06,339 INFO internal.SharedState: Warehouse path is 'file:/usr/spark-2.2.0/spark-warehouse'.
2020-08-19 13:32:06,343 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@4a11ef03{/SQL,null,AVAILABLE,@Spark}
2020-08-19 13:32:06,343 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@1f2ec360{/SQL/json,null,AVAILABLE,@Spark}
2020-08-19 13:32:06,344 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@29289560{/SQL/execution,null,AVAILABLE,@Spark}
2020-08-19 13:32:06,344 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@62f10152{/SQL/execution/json,null,AVAILABLE,@

2020-08-19 13:32:20,999 INFO datasources.FileSourceStrategy: Pruning directories with: 
2020-08-19 13:32:20,999 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2020-08-19 13:32:21,000 INFO datasources.FileSourceStrategy: Output Data Schema: struct<sex: string, length: double, diameter: double, height: double, whole_weight: double ... 7 more fields>
2020-08-19 13:32:21,000 INFO execution.FileSourceScanExec: Pushed Filters: 
2020-08-19 13:32:21,092 INFO codegen.CodeGenerator: Code generated in 50.560046 ms
2020-08-19 13:32:21,110 INFO memory.MemoryStore: Block broadcast_7 stored as values in memory (estimated size 444.5 KB, free 364.7 MB)
2020-08-19 13:32:21,124 INFO memory.MemoryStore: Block broadcast_7_piece0 stored as bytes in memory (estimated size 41.5 KB, free 364.6 MB)
2020-08-19 13:32:21,125 INFO storage.BlockManagerInfo: Added broadcast_7_piece0 in memory on 10.0.126.24:33621 (size: 41.5 KB, free: 366.1 MB)
2020-08-19 13:32:21,126 INFO spark.SparkContext: Created broadc

In [11]:
print('Top 5 rows from s3://{}/{}/train/'.format(bucket, input_preprocessed_prefix))
!aws s3 cp --quiet s3://$bucket/$input_preprocessed_prefix/train/part-00000 - | head -n5

Top 5 rows from s3://sagemaker-us-east-1-452432741922/sagemaker/spark-preprocess-demo/2020-08-18-22-44-22/input/preprocessed/abalone/train/
5.0,0.0,0.0,0.275,0.195,0.07,0.08,0.031,0.0215,0.025
6.0,0.0,0.0,0.29,0.21,0.075,0.275,0.113,0.0675,0.035
7.0,0.0,0.0,0.305,0.225,0.07,0.1485,0.0585,0.0335,0.045
9.0,0.0,0.0,0.33,0.26,0.08,0.2,0.0625,0.05,0.07
6.0,0.0,0.0,0.335,0.22,0.07,0.17,0.076,0.0365,0.05


In [12]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sagemaker_session.boto_region_name, 'xgboost', repo_version="0.90-1")
print(training_image)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
	get_image_uri(region, 'xgboost', '1.0-1').


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3


In [54]:
training_image

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3'

In [55]:
role

'arn:aws:iam::452432741922:role/workshop-role'

In [16]:
s3_train_data = 's3://{}/{}/{}'.format(bucket, input_preprocessed_prefix, 'train/part')
s3_validation_data = 's3://{}/{}/{}'.format(bucket, input_preprocessed_prefix, 'validation/part')
s3_output_location = 's3://{}/{}/{}'.format(bucket, prefix, 'xgboost_model')

xgb_model = sagemaker.estimator.Estimator(training_image,
                                          role, 
                                          train_instance_count=1, 
                                          train_instance_type='ml.m4.xlarge',
                                          train_volume_size = 20,
                                          train_max_run = 3600,
                                          input_mode= 'File',
                                          output_path=s3_output_location,
                                          sagemaker_session=sagemaker_session)

xgb_model.set_hyperparameters(objective = "reg:linear",
                              eta = .2,
                              gamma = 4,
                              max_depth = 5,
                              num_round = 10,
                              subsample = 0.7,
                              silent = 0,
                              min_child_weight = 6)

train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/csv', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/csv', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

In [79]:
xgb_model

In [68]:
s3_train_data

's3://sagemaker-us-east-1-452432741922/sagemaker/spark-preprocess-demo/2020-08-18-22-44-22/input/preprocessed/abalone/train/part'

In [64]:
train_data.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://sagemaker-us-east-1-452432741922/sagemaker/spark-preprocess-demo/2020-08-18-22-44-22/input/preprocessed/abalone/train/part',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'text/csv'}

In [17]:
xgb_model.fit(inputs=data_channels, logs=True)

2020-08-19 14:03:08 Starting - Starting the training job...
2020-08-19 14:03:14 Starting - Launching requested ML instances.........
2020-08-19 14:04:56 Starting - Preparing the instances for training......
2020-08-19 14:06:09 Downloading - Downloading input data
2020-08-19 14:06:09 Training - Downloading the training image.....INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[14:06:44] 3339x9 matrix with 30051 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter of CSV

In [34]:
import json
schema = {
    "input": [
        {
            "name": "sex",
            "type": "string"
        }, 
        {
            "name": "length",
            "type": "double"
        }, 
        {
            "name": "diameter",
            "type": "double"
        }, 
        {
            "name": "height",
            "type": "double"
        }, 
        {
            "name": "whole_weight",
            "type": "double"
        }, 
        {
            "name": "shucked_weight",
            "type": "double"
        },
        {
            "name": "viscera_weight",
            "type": "double"
        }, 
        {
            "name": "shell_weight",
            "type": "double"
        }, 
    ],
    "output": 
        {
            "name": "features",
            "type": "double",
            "struct": "vector"
        }
}
schema_json = json.dumps(schema)
print(schema_json)

{"input": [{"name": "sex", "type": "string"}, {"name": "length", "type": "double"}, {"name": "diameter", "type": "double"}, {"name": "height", "type": "double"}, {"name": "whole_weight", "type": "double"}, {"name": "shucked_weight", "type": "double"}, {"name": "viscera_weight", "type": "double"}, {"name": "shell_weight", "type": "double"}], "output": {"name": "features", "type": "double", "struct": "vector"}}


In [71]:
xgb_model.model_data

's3://sagemaker-us-east-1-452432741922/sagemaker/spark-preprocess-demo/2020-08-18-22-44-22/xgboost_model/sagemaker-xgboost-2020-08-19-14-03-08-551/output/model.tar.gz'

In [72]:
s3_output_location

's3://sagemaker-us-east-1-452432741922/sagemaker/spark-preprocess-demo/2020-08-18-22-44-22/xgboost_model'

In [74]:
dir(xgb_model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_compilation_job_config',
 '_create_sagemaker_model',
 '_enable_network_isolation',
 '_framework',
 '_get_framework_version',
 '_inferentia_image',
 '_init_sagemaker_session_if_does_not_exist',
 '_is_compiled_model',
 '_model_name',
 '_neo_image',
 '_neo_image_account',
 'check_neo_region',
 'compile',
 'delete_model',
 'deploy',
 'enable_network_isolation',
 'endpoint_name',
 'env',
 'image',
 'model_data',
 'model_kms_key',
 'name',
 'predictor_cls',
 'prepare_container_def',
 'role',
 'sagemaker_session',
 'transformer',
 'vpc_config']

In [75]:
mleap_model_prefix

'sagemaker/spark-preprocess-demo/automated-job/mleap-model'

In [76]:
bucket

'sagemaker-us-east-1-452432741922'

In [35]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
from sagemaker.sparkml.model import SparkMLModel

sparkml_data = 's3://{}/{}/{}'.format(bucket, mleap_model_prefix, 'model.tar.gz')
# passing the schema defined above by using an environment variable that sagemaker-sparkml-serving understands
sparkml_model = SparkMLModel(model_data=sparkml_data, env={'SAGEMAKER_SPARKML_SCHEMA' : schema_json})
xgb_model = Model(model_data=xgb_model.model_data, image=training_image)

model_name = 'inference-pipeline-' + timestamp_prefix
sm_model = PipelineModel(name=model_name, role=role, models=[sparkml_model, xgb_model])

In [ ]:
# endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix
# sm_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

In [47]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
# payload = "F,0.515,0.425,0.14,0.766,0.304,0.1725,0.255"
# predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sagemaker_session, serializer=csv_serializer,
#                                 content_type=CONTENT_TYPE_CSV, accept=CONTENT_TYPE_CSV)
# print(predictor.predict(payload))

In [48]:
# payload = {"data": ["F",0.515,0.425,0.14,0.766,0.304,0.1725,0.255]}
# predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sagemaker_session, serializer=json_serializer,
#                                 content_type=CONTENT_TYPE_JSON, accept=CONTENT_TYPE_CSV)

# print(predictor.predict(payload))

BATCH TRANSFORM

In [49]:
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/batch_input_abalone.csv
!printf "\n\nShowing first five lines\n\n"    
!head -n 5 batch_input_abalone.csv 
!printf "\n\nAs we can see, it is identical to the training file apart from the label being absent here.\n\n"  

--2020-08-19 15:08:31--  https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/batch_input_abalone.csv
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.218.208
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.218.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 654 [text/csv]
Saving to: ‘batch_input_abalone.csv.1’

batch_input_abalone 100%[===================>]     654  --.-KB/s    in 0s      

2020-08-19 15:08:31 (19.9 MB/s) - ‘batch_input_abalone.csv.1’ saved [654/654]



Showing first five lines

M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15
M,0.35,0.265,0.09,0.2255,0.0995,0.0485,0.07
F,0.53,0.42,0.135,0.677,0.2565,0.1415,0.21
M,0.44,0.365,0.125,0.516,0.2155,0.114,0.155
I,0.33,0.255,0.08,0.205,0.0895,0.0395,0.055


As we can see, it is identical to the training file apart from the label being absent here.



In [50]:
 = sagemaker_session.upload_data(path='batch_input_abalone.csv', bucket=bucket, key_prefix='batch')batch_input_loc

In [78]:
role

'arn:aws:iam::452432741922:role/workshop-role'

In [52]:
transformer = sm_model.transformer(1,'ml.m4.xlarge')

In [53]:
input_data_path = 's3://{}/{}/{}'.format(bucket, 'batch', 'batch_input_abalone.csv')
output_data_path = 's3://{}/{}/{}'.format(bucket, 'batch_output/abalone', timestamp_prefix)
job_name = 'serial-inference-batch-' + timestamp_prefix
transformer = sagemaker.transformer.Transformer(
    # This was the model created using PipelineModel and it contains feature processing and XGBoost
    model_name = model_name,
    instance_count = 1,
    instance_type = 'ml.m4.xlarge',
    strategy = 'SingleRecord',
    assemble_with = 'Line',
    output_path = output_data_path,
    base_transform_job_name='serial-inference-batch',
    sagemaker_session=sagemaker_session,
    accept = CONTENT_TYPE_CSV
)
transformer.transform(data = input_data_path,
                      job_name = job_name,
                      content_type = CONTENT_TYPE_CSV, 
                      split_type = 'Line')
transformer.wait()

....................................  .   ____          _            __ _ _
 /\\ / ___'_ __ _ _(_)_ __  __ _ \ \ \ \
( ( )\___ | '_ | '_| | '_ \/ _` | \ \ \ \
 \\/  ___)| |_)| | | | | || (_| |  ) ) ) )
  '  |____| .__|_| |_|_| |_\__, | / / / /
 =========|_|==============|___/=/_/_/_/
 :: Spring Boot ::                  (v2.2)

2020-08-19 15:15:28.636  INFO 7 --- [           main] com.amazonaws.sagemaker.App              : Starting App v2.2 on b95ecebd125b with PID 7 (/usr/local/lib/sparkml-serving-2.2.jar started by root in /sagemaker-sparkml-model-server)
2020-08-19 15:15:28.642  INFO 7 --- [           main] com.amazonaws.sagemaker.App              : No active profile set, falling back to default profiles: default
  .   ____          _            __ _ _
 /\\ / ___'_ __ _ _(_)_ __  __ _ \ \ \ \
( ( )\___ | '_ | '_| | '_ \/ _` | \ \ \ \
 \\/  ___)| |_)| | | | | || (_| |  ) ) ) )
  '  |____| .__|_| |_|_| |_\__, | / / / /
 =========|_|==============|___/=/_/_/_/
 :: Spring Boot ::        

In [58]:
output_data_path

's3://sagemaker-us-east-1-452432741922/batch_output/abalone/automated-job'

In [60]:
s3 = boto3.resource('s3')

KEY = 'batch_output/abalone/automated-job/batch_input_abalone.csv.out'
s3.Bucket(bucket).download_file(KEY, 'batch_output_abalone.csv')

!head batch_output_abalone.csv

8.222951889038086
7.495278835296631
10.074289321899414
8.880287170410156
5.981818675994873
7.115278244018555
13.197749137878418
11.355527877807617
9.285648345947266
11.386305809020996
